In [1]:
import praw
import pandas as %pd
import datetime as dt
from collections import defaultdict 

In [2]:
# client_id='PERSONAL_USE_SCRIPT_14_CHARS', 
#                      client_secret='SECRET_KEY_27_CHARS ', 
#                      user_agent='YOUR_APP_NAME'
reddit = praw.Reddit(client_id='R30fcxAZLJ7Wyw',
                     client_secret='8Zf87krHYa_zXRXPWEF8hK2Twfs',
                     user_agent='project1',
                     username = 'gbatra',
                     password = "HOLYMOLY1999"
                    )
print(reddit.read_only)

False


In [3]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [4]:
india_subreddit = reddit.subreddit('india')
print(india_subreddit.description)

### We are looking for additional moderators. If you believe you can help, apply [via modmail](https://www.reddit.com/message/compose?to=%2Fr%2Findia)

###[r/India Rules](https://www.reddit.com/r/India/wiki/rules)

###[PushBullet Notifications](https://www.pushbullet.com/channel?tag=redditindia)

###[r/India Discord Server](https://discord.gg/6V29X7f)

###[Scheduled threads](https://www.reddit.com/r/india/comments/b27sx9/scheduled_threads/)

###[r/India twitter](https://twitter.com/redditindia)

#Filters & Hot Topics:

> **Enable**[All](http://www.reddit.com/r/india/#aa)[Non-Political](http://yy.reddit.com/r/india/#yy)[Politics](http://xx.reddit.com/r/india/#xx)[\[R\]eddiquette](http://zz.reddit.com/r/india/#zz)[AMA](https://am.reddit.com/r/india/search?q=flair%3AAMA&sort=new&restrict_sr=on/#am)[Scheduled](https://sd.reddit.com/r/india/search?q=flair%3AScheduled+-title%3ARandom&restrict_sr=on&sort=new&t=all/#sd)[Science & Tech.](http://bb.reddit.com/r/india/#bb)[Self Posts](http://ss.r

In [5]:
# Each subreddit has five different ways of organizing the topics created by redditors: .hot, .new,
# .controversial, .top, and .gilded. 
# You can also use .search("SEARCH_KEYWORDS") to get only results matching an engine search.

In [6]:
# get 10 hot posts from the MachineLearning subreddit
hot_posts = india_subreddit.hot(limit=10)

In [7]:
# test = []
# for post in hot_posts:
#     print("Title: "+ post.title)
#     print("Score: "+ str(post.score))
#     print("ID: "+ str(post.id))
#     print("Subreddit: "+ str(post.subreddit))
#     print("URL: "+ post.url)
#     print("No. of comments: "+ str(post.num_comments))
#     print("Self Text: "+ post.selftext)
#     print("Post Created: "+ str(post.created))
#     test.append([post.title, post.score, post.id])
#     print()
# test

In [8]:
# india_posts = defaultdict(list)
# for post in hot_posts:
#     india_posts['title'].append(post.title)
#     india_posts['score'].append(post.score)
#     india_posts['url'].append(post.url)
#     india_posts['num_comments'].append(post.num_comments)
#     india_posts['text'].append(post.selftext)
#     india_posts['created'].append(post.created)
#     #  a limit of None means that all MoreComments objects will be replaced until there are none left
#     for comment in post.comments:
#         india_posts['comments'].append(comment)

# india_posts

# This gave out an error ' ValueError: arrays must all be same length' when being converted to Dataframe

In [9]:
india_posts = defaultdict(list)
for post in hot_posts:
    india_posts['title'].append(post.title)
    india_posts['score'].append(post.score)
    india_posts['url'].append(post.url)
    india_posts['num_comments'].append(post.num_comments)
    india_posts['text'].append(post.selftext)
    india_posts['created'].append(post.created)
    #  a limit of None means that all MoreComments objects will be replaced until there are none left
    post.comments.replace_more(limit=None)
    all_comments = ''
    for comment in post.comments:
        all_comments = all_comments + ' ' + comment.body
    
    india_posts['comments'].append(all_comments)

india_posts

defaultdict(list,
            {'title': ['Coronavirus (COVID-19) Megathread - News and Updates - 4',
              '[Monthly Happiness Thread] Randians, please share a good/positive/happy news happening in your life recently ! :)',
              '#OnThisDay in 1975, history was made when India’s first satellite - Aryabhata, was launched from Kapustin Yar in Russia.',
              'Different stages of hair loss in perfect order. Mumbai local train memories.',
              "Tejasvi Surya deletes tweet on Arab women that read 'they never had orgasm in the last few hundred years'",
              'Pregnant woman in Bengaluru walks 5 km in search of hospital, doctor couple save baby',
              'All the 186 COVID-19 positive cases that were reported yesterday were asymptomatic, they didn’t know they had Coronavirus. This is more worrying: Delhi CM Arvind Kejriwal',
              'Saudi scholar calls for expulsion of Hindu supremacists from the Gulf',
              'Rajasthan Congress M

In [10]:
india_postsdf = pd.DataFrame(india_posts)

In [11]:
_timestamp = india_postsdf["created"].apply(get_date)
india_postsdf = india_postsdf.assign(timestamp = _timestamp)
del india_postsdf['created']

In [25]:
# preprocess the title, comments and text
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess(text):

    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            if w.isalnum():
                w = w.lower()
                filtered_sentence.append(w)
    return(filtered_sentence)

In [28]:
india_postsdf['title'] = india_postsdf['title'].apply(preprocess)

In [26]:
india_postsdf['comments'] = india_postsdf['comments'].apply(preprocess)

In [30]:
india_postsdf['text'] = india_postsdf['text'].apply(preprocess)

In [32]:
india_postsdf.to_csv('india_posts.csv', index=False)

In [33]:
india_posts = pd.read_csv('india_posts.csv')
india_posts.head(10)

,title,score,url,num_comments,text,comments,timestamp
0,"['coronavirus', 'megathread', 'news', 'updates...",117,https://www.reddit.com/r/india/comments/g1zi21...,1483,"['fundraisers', 'donation', 'links', 'https', ...","['fundraisers', 'donation', 'links', 'https', ...",2020-04-16 09:30:15
1,"['monthly', 'happiness', 'thread', 'randians',...",0,https://www.reddit.com/r/india/comments/g4d2ix...,2,"['3', 'links', 'feb', 'mht', 'https', 'jan', '...","['found', 'chicken', 'nuggets', 'today', 'loca...",2020-04-20 08:20:40
2,"['onthisday', '1975', 'history', 'made', 'indi...",2268,https://i.redd.it/8guugveh2qt41.jpg,71,[],"['many', 'leaders', 'beheaded', 'sweet', 'cont...",2020-04-19 20:23:24
3,"['different', 'stages', 'hair', 'loss', 'perfe...",2519,https://i.redd.it/ydbmwsa7jpt41.jpg,78,[],"['so', 'i', 'guess', 'bald', 'shakaal', 'hey',...",2020-04-19 18:36:00
4,"['tejasvi', 'surya', 'deletes', 'tweet', 'arab...",161,https://www.ibtimes.co.in/tejasvi-surya-delete...,21,[],"['tejasvi', 'surya', 'deletes', 'tweet', 'as',...",2020-04-20 04:40:01
5,"['pregnant', 'woman', 'bengaluru', 'walks', '5...",96,https://www.thenewsminute.com/article/pregnant...,13,[],"['lock', 'may', 'kill', 'much', 'people', 'cor...",2020-04-20 06:38:42
6,"['all', '186', 'positive', 'cases', 'reported'...",599,https://twitter.com/ANI/status/125176181030855...,94,[],"['so', 'could', 'chances', 'get', 'completely'...",2020-04-19 20:13:23
7,"['saudi', 'scholar', 'calls', 'expulsion', 'hi...",244,https://www.middleeastmonitor.com/20200417-sau...,61,[],"['what', 'goes', 'around', 'comes', 'around', ...",2020-04-20 00:46:55
8,"['rajasthan', 'congress', 'mla', 'took', 'back...",327,https://twitter.com/IndiaToday/status/12517392...,17,[],"['this', 'chomu', 'must', 'know', 'ration', 'p...",2020-04-19 21:20:41
9,"['tvf', 'panchayat', 'authentic', 'rural', 'in...",103,https://www.reddit.com/r/india/comments/g48akq...,27,"['i', 'wo', 'deny', 'i', 'seen', 'many', 'rura...","['i', 'seen', 'last', 'night', 'the', 'people'...",2020-04-20 03:41:44


In [35]:
india_posts.dtypes

title           object
score            int64
url             object
num_comments     int64
text            object
comments        object
timestamp       object
dtype: object

In [ ]:
# india_posts['title'] = pd.Series(india_posts['title'], dtype='string')
# india_posts.loc[:,'title'] = india_posts.loc[:, 'title'].astype(str)
# india_posts['title'].apply(str)
# india_posts['title'] = india_posts['title'].astype('|S')
# india_posts['title']

In [36]:
flairs=['Politics','Non-Political','AskIndia', "[R]eddiquette", 'Science/Technology', 'Policy/Economy', 'Business/Finance', 'Scheduled', 'Sports', 'Food', 'Photography', 'AMA', ]

In [ ]:
data = defaultdict(list)
for flair in flairs:
    search_flair = india_subreddit.search(flair, limit=100)
    for post in search_flair:
        data['flair'].append(flair)
        data['title'].append(post.title)
        data['score'].append(post.score)
        data['url'].append(post.url)
        data['num_comments'].append(post.num_comments)
        data['text'].append(post.selftext)
        data['created'].append(post.created)
        post.comments.replace_more(limit=None)
        all_comments = ''
        for comment in post.comments:
            all_comments = all_comments + ' ' + comment.body

        data['comments'].append(all_comments)
data

In [ ]:
datadf = pd.DataFrame(data)
_timestamp = datadf['created'].apply(get_date)
datadf = datadf.assign(timestamp = _timestamp)
del datadf['created']

datadf['title'] = datadf['title'].apply(preprocess)
datadf['comments'] = datadf['comments'].apply(preprocess)
datadf['text'] = datadf['text'].apply(preprocess)

In [ ]:
datadf.to_csv('dataset.csv', index=False)

In [ ]:
len(datadf['title'])

In [ ]:
datadf = pd.read_csv('dataset.csv')
datadf.head(10)

In [ ]:
# Common text classification algorithms : 
# logistic regression, support vector machine, naive Bayes classifier, decision trees, random forest, K nearest neighbor